# Note this model excluded regio1, postalcodes, firingtypes and conditions

they didn't have much influence, since we have cities
interesting about heating type though

In [1]:
import pandas as pd
import seaborn as sns
#the file is a number of data from ImmoScout (webscrapped for all Germany)
file_path = 'https://raw.githubusercontent.com/okravtsova123/ironhack_study/main/MidTermProject/rent.csv'

# reading file with different encodings
try:
    df = pd.read_csv(file_path, encoding='utf-8', sep=';')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding='latin-1',  sep=';')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='ISO-8859-1',  sep=';')


# Cleanng and preparing data

In [2]:
#checking the df
df.head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,...,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,energyEfficiencyClass
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,...,ground_floor,44269,4.0,181.4,1.0,3.0,4,True,4,NaN
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,...,ground_floor,67459,3.0,NaN,NaN,NaN,3,False,4,NaN
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,...,apartment,1097,3.0,NaN,3.0,4.0,3,False,4,NaN
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,...,other,9599,3.0,86.0,3.0,NaN,3,False,2,NaN
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,...,apartment,28213,3.0,188.9,1.0,NaN,3,False,4,NaN


In [3]:
# Performing additional formatting for column names (lowercase, strip and replace spaces with underscores) - esier to work further
df.columns = [name.strip().replace(" ", "_").lower() for name in df.columns]

In [4]:
df.columns

Index(['regio1', 'servicecharge', 'heatingtype', 'telekomtvoffer',
       'telekomhybriduploadspeed', 'newlyconst', 'balcony', 'picturecount',
       'pricetrend', 'telekomuploadspeed', 'totalrent', 'yearconstructed',
       'scoutid', 'noparkspaces', 'firingtypes', 'haskitchen', 'geo_bln',
       'cellar', 'yearconstructedrange', 'baserent', 'housenumber',
       'livingspace', 'geo_krs', 'condition', 'interiorqual', 'petsallowed',
       'street', 'streetplain', 'lift', 'baserentrange', 'typeofflat',
       'geo_plz', 'norooms', 'thermalchar', 'floor', 'numberoffloors',
       'noroomsrange', 'garden', 'livingspacerange', 'energyefficiencyclass'],
      dtype='object')

In [5]:
#function to check null values
def null_check(df):
    print(f'Total null values per row: \n{df.isnull().sum(axis=1)}\n')
    print(f'Total null values per column: \n{df.isnull().sum()}\n')

In [6]:
#applying
null_check(df)

Total null values per row: 
0         3
1         8
2         4
3         8
4         8
         ..
268845    7
268846    7
268847    3
268848    4
268849    9
Length: 268850, dtype: int64

Total null values per column: 
regio1                           0
servicecharge                 6909
heatingtype                  44856
telekomtvoffer               32619
telekomhybriduploadspeed    223830
newlyconst                       0
balcony                          0
picturecount                     0
pricetrend                    1832
telekomuploadspeed           33358
totalrent                    40517
yearconstructed              57045
scoutid                          0
noparkspaces                175798
firingtypes                  56964
haskitchen                       0
geo_bln                          0
cellar                           0
yearconstructedrange         57045
baserent                         0
housenumber                  71018
livingspace                      0
geo_krs  

In [7]:
#function to find duplicates
def dup_check(df):
    print(f'Duplicates found: {df.duplicated().any()}\n')
    print(f'Number of duplicates: {df.duplicated().sum()}\n')

In [8]:
dup_check(df)

Duplicates found: False

Number of duplicates: 0



In [9]:
#Choosing only new buildings as rent is not controlled by government
df = df[(df['newlyconst']== True)]

In [10]:
df["geo_krs"].nunique()

402

In [11]:
#choosing the facors for the model
df = df.loc[:, ['baserent', 'heatingtype', 'balcony', 'noparkspaces', 'haskitchen',
       'cellar','livingspace', 'petsallowed', 'geo_krs', 'lift', 'norooms', 'floor', 'garden', 'energyefficiencyclass']]


In [12]:
df

,baserent,heatingtype,balcony,noparkspaces,haskitchen,cellar,livingspace,petsallowed,geo_krs,lift,norooms,floor,garden,energyefficiencyclass
2,965.00,floor_heating,True,1.0,False,True,83.80,NaN,Dresden,True,3.0,3.0,False,NaN
14,972.60,gas_heating,True,1.0,False,True,87.00,negotiable,Dortmund,True,3.0,0.0,False,NaN
29,520.08,district_heating,True,1.0,False,True,47.28,NaN,Schwerin,True,2.0,NaN,False,NaN
48,870.00,NaN,True,NaN,True,False,81.00,NaN,Steinburg_Kreis,False,3.0,2.0,False,B
69,900.00,floor_heating,True,1.0,False,True,74.80,NaN,Dresden,True,3.0,0.0,False,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268806,2200.00,district_heating,True,1.0,True,True,169.00,negotiable,Mannheim,True,4.0,1.0,True,NaN
268809,371.43,heat_pump,True,1.0,False,True,59.38,no,Rheinisch_Bergischer_Kreis,True,2.0,NaN,False,NaN
268821,680.00,central_heating,True,1.0,False,True,79.50,no,Rhein_Hunsr?ck_Kreis,False,2.0,1.0,False,NaN
268827,990.00,central_heating,True,1.0,True,False,89.00,no,Hannover_Kreis,True,2.5,1.0,False,NaN


In [13]:
df = df.reset_index()

In [14]:
df

,index,baserent,heatingtype,balcony,noparkspaces,haskitchen,cellar,livingspace,petsallowed,geo_krs,lift,norooms,floor,garden,energyefficiencyclass
0,2,965.00,floor_heating,True,1.0,False,True,83.80,NaN,Dresden,True,3.0,3.0,False,NaN
1,14,972.60,gas_heating,True,1.0,False,True,87.00,negotiable,Dortmund,True,3.0,0.0,False,NaN
2,29,520.08,district_heating,True,1.0,False,True,47.28,NaN,Schwerin,True,2.0,NaN,False,NaN
3,48,870.00,NaN,True,NaN,True,False,81.00,NaN,Steinburg_Kreis,False,3.0,2.0,False,B
4,69,900.00,floor_heating,True,1.0,False,True,74.80,NaN,Dresden,True,3.0,0.0,False,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21166,268806,2200.00,district_heating,True,1.0,True,True,169.00,negotiable,Mannheim,True,4.0,1.0,True,NaN
21167,268809,371.43,heat_pump,True,1.0,False,True,59.38,no,Rheinisch_Bergischer_Kreis,True,2.0,NaN,False,NaN
21168,268821,680.00,central_heating,True,1.0,False,True,79.50,no,Rhein_Hunsr?ck_Kreis,False,2.0,1.0,False,NaN
21169,268827,990.00,central_heating,True,1.0,True,False,89.00,no,Hannover_Kreis,True,2.5,1.0,False,NaN


In [15]:
null_check(df)

Total null values per row: 
0        2
1        1
2        3
3        3
4        1
        ..
21166    1
21167    2
21168    1
21169    1
21170    1
Length: 21171, dtype: int64

Total null values per column: 
index                        0
baserent                     0
heatingtype               3045
balcony                      0
noparkspaces              6611
haskitchen                   0
cellar                       0
livingspace                  0
petsallowed               9357
geo_krs                      0
lift                         0
norooms                      0
floor                     5238
garden                       0
energyefficiencyclass    11456
dtype: int64



In [16]:
df = df.dropna()

In [17]:
#checking unqiue values in hitting
df['heatingtype'].unique()

array(['floor_heating', 'central_heating', 'district_heating',
       'heat_pump', 'combined_heat_and_power_plant', 'gas_heating',
       'wood_pellet_heating', 'self_contained_central_heating',
       'solar_heating', 'stove_heating', 'oil_heating',
       'electric_heating'], dtype=object)

In [18]:
#checking unqiue values in coditions
#df['condition'].unique()

In [19]:
#checking unqiue values in coditions
#df['firingtypes'].nunique()

In [20]:
#unifying first time use and all refurbishement, modernization and renovation in 1 thing

#def condition_clean(state):
#        if 'first_time_use' in state:
#           state='first_time_use'
#        elif state=='fully_renovated' or state=='refurbished' or state=='modernized':
#           state='refurbished'
#       else:
#           state=state
#       return state
    
#unifying types of firing

def firingtypes_clean(ftype):
    if 'gas' in ftype:
        ftype='gas'
    elif 'electricity' in ftype:
        ftype='electricity'
    elif 'solar_heating' in ftype or 'wind_energy' in ftype or'hydro_energy' in ftype or 'bio_energy' in ftype or 'geothermal' in ftype or 'thermal_energy' in ftype or 'renewable_energy' in ftype or 'regenerative_energy' in ftype:
        ftype='renewable_energy_heating'
    elif "pellet_heating" in ftype or 'fossil_fuels' in ftype or 'wood_chips' in ftype:
        ftype='pellet_heating'
    elif 'district_heating' in ftype:
        ftype='district_heating'
    else:
        ftype=ftype
    return ftype

In [21]:
#applying defined function for cleaning    
#df['condition']=df['condition'].apply(condition_clean)
#df['firingtypes']=df['firingtypes'].apply(firingtypes_clean)

In [22]:
#checking energyefficiencyclass
print(df["energyefficiencyclass"].groupby(df["energyefficiencyclass"]).count())

#droping rows with NO_INFORMATION
df=df[df["energyefficiencyclass"]!='NO_INFORMATION'].reset_index()

energyefficiencyclass
A                 908
A_PLUS            861
B                 929
C                 161
D                  37
E                   3
F                   2
NO_INFORMATION    485
Name: energyefficiencyclass, dtype: int64


In [23]:
#cleaning two useless columns. And heatng type is removed as it's a copy of firing type
columns_needed=['baserent', 'balcony', 'noparkspaces', 'haskitchen',
       'cellar','livingspace', 'petsallowed', 'geo_krs', 'lift', 'norooms', 'floor', 'garden', 'energyefficiencyclass']
df=df.loc[: , columns_needed]

In [24]:
#cheking spelling mistakes in regio1
#df['regio1'].groupby(df['regio1']).count()

In [25]:
#clening misspellings

def regio1_clean(region):
    if region=='Baden_W?rttemberg':
        region='Baden-Württemberg'
    elif region=='Th?ringen':
        region='Thüringia'
    else:
        region=region
    return region

In [26]:
#applying
#df['regio1']=df['regio1'].apply(regio1_clean)

In [27]:
#cheking spelling mistakes in geo_krs
#REMOVE cities with less than 10 appartments, because they give too much influence in model
cities = df.groupby('geo_krs').size().reset_index(name='count')
cities_to_drop = cities[cities['count'] <= 30]
df = df[~df['geo_krs'].isin(cities_to_drop['geo_krs'])].reset_index(drop=True)
df['geo_krs'].groupby(df['geo_krs']).count()

geo_krs
Berlin                   127
Bonn                      32
D?sseldorf                50
Dresden                  142
Frankfurt_am_Main        119
G?tersloh_Kreis           34
Hamburg                   51
Leipzig                  185
M?hldorf_am_Inn_Kreis     32
M?nchen                  143
Mannheim                  56
Name: geo_krs, dtype: int64

In [28]:
#checking pets
df['petsallowed'].groupby(df['petsallowed']).count()

petsallowed
negotiable    551
no            259
yes           161
Name: petsallowed, dtype: int64

In [29]:
#checking outliers - identifying IQR

df['baserent'].describe()
IQR=df['baserent'].quantile(0.75)-df['baserent'].quantile(0.25)
base_min=df['baserent'].quantile(0.25)-1.5*IQR
base_max=df['baserent'].quantile(0.75)+3*IQR
print(base_min, base_max)

#countring how many flats are outliers
x_low=0
x_high=0
for price in df['baserent']:
    if price<base_min:
        x_low+=1
    elif price>base_max:
        x_high+=1

print(f'outliers too low:{x_low}\noutliers too high: {x_high}')

-32.412499999999795 3371.8949999999995
outliers too low:0
outliers too high: 19


In [30]:
#dropping rows with outliers
df=df.loc[df['baserent']<base_max]

In [31]:
df.isna().sum()

baserent                 0
balcony                  0
noparkspaces             0
haskitchen               0
cellar                   0
livingspace              0
petsallowed              0
geo_krs                  0
lift                     0
norooms                  0
floor                    0
garden                   0
energyefficiencyclass    0
dtype: int64

# ENCODING FACTORS FOR MODEL CREATION

In [32]:
#splitting df into numericals and categoricals
categoricals = df.select_dtypes(include=[object, bool])
numericals = df.select_dtypes(include = 'number')

In [33]:
categoricals

,balcony,haskitchen,cellar,petsallowed,geo_krs,lift,garden,energyefficiencyclass
0,True,True,True,no,M?nchen,True,False,A
1,True,True,True,yes,Dresden,True,False,C
2,True,False,True,negotiable,Leipzig,True,False,B
3,True,True,True,yes,Leipzig,True,True,A
4,True,False,True,negotiable,D?sseldorf,True,False,B
...,...,...,...,...,...,...,...,...
966,True,True,False,negotiable,Leipzig,True,False,B
967,True,False,True,yes,Dresden,True,False,B
968,True,False,True,no,Leipzig,True,False,B
969,True,True,False,yes,Dresden,True,False,C


In [34]:
numericals

,baserent,noparkspaces,livingspace,norooms,floor
0,1680.00,1.0,91.00,1.5,2.0
1,1337.22,1.0,116.28,3.0,1.0
2,1321.61,1.0,104.89,4.0,4.0
3,1744.26,1.0,130.00,3.0,8.0
4,890.00,1.0,64.40,2.0,4.0
...,...,...,...,...,...
966,1115.00,1.0,103.14,4.0,3.0
967,1217.00,1.0,109.10,4.0,2.0
968,1159.36,2.0,144.92,4.0,1.0
969,1533.92,1.0,121.74,5.0,2.0


In [35]:
ordinals = categoricals[["energyefficiencyclass", "balcony","haskitchen", "cellar", "lift", "garden"]].reset_index()
nominals = categoricals.drop(columns=["energyefficiencyclass", "balcony","haskitchen", "cellar", "lift", "garden"]).reset_index()

In [36]:
nominals=nominals.drop('index', axis=1)
nominals

,petsallowed,geo_krs
0,no,M?nchen
1,yes,Dresden
2,negotiable,Leipzig
3,yes,Leipzig
4,negotiable,D?sseldorf
...,...,...
947,negotiable,Leipzig
948,yes,Dresden
949,no,Leipzig
950,yes,Dresden


In [37]:
#checking how we will encode the ordinals
ordinals["energyefficiencyclass"].groupby(df["energyefficiencyclass"]).count()

energyefficiencyclass
A         221
A_PLUS    143
B         460
C          97
D          12
Name: energyefficiencyclass, dtype: int64

In [38]:
from sklearn.preprocessing import OrdinalEncoder

In [39]:
#encoding energy efficiency class for model creation

enc = OrdinalEncoder(categories=[["A_PLUS", "A", "B", "C","D", "E", "F"]])

ordinals["energyefficiencyclass"] = enc.fit_transform(ordinals[["energyefficiencyclass"]])

In [40]:
#encoding haskitchen for model creation

ordinals["haskitchen"] = ordinals["haskitchen"].astype(object)
enc = OrdinalEncoder(categories=[[True, False]])
ordinals["haskitchen"] = enc.fit_transform(ordinals[["haskitchen"]])

In [41]:
#encoding balcony for model creation

ordinals["balcony"] = ordinals["balcony"].astype(object)
enc = OrdinalEncoder(categories=[[True, False]])
ordinals["balcony"] = enc.fit_transform(ordinals[["balcony"]])

In [42]:
#encoding cellar for model creation

ordinals["cellar"] = ordinals["cellar"].astype(object)
enc = OrdinalEncoder(categories=[[True, False]])
ordinals["cellar"] = enc.fit_transform(ordinals[["cellar"]])

In [43]:
#encoding lift for model creation

ordinals["lift"] = ordinals["lift"].astype(object)
enc = OrdinalEncoder(categories=[[True, False]])
ordinals["lift"] = enc.fit_transform(ordinals[["lift"]])

In [44]:
#encoding garden for model creation

ordinals["garden"] = ordinals["garden"].astype(object)
enc = OrdinalEncoder(categories=[[True, False]])
ordinals["garden"] = enc.fit_transform(ordinals[["garden"]])

In [45]:
ordinals.isna().sum()

index                    0
energyefficiencyclass    0
balcony                  0
haskitchen               0
cellar                   0
lift                     0
garden                   0
dtype: int64

In [46]:
#making dunnies from nominals
nominals = pd.get_dummies(nominals, dtype=int)

In [47]:
nominals.isna().sum()

petsallowed_negotiable           0
petsallowed_no                   0
petsallowed_yes                  0
geo_krs_Berlin                   0
geo_krs_Bonn                     0
geo_krs_D?sseldorf               0
geo_krs_Dresden                  0
geo_krs_Frankfurt_am_Main        0
geo_krs_G?tersloh_Kreis          0
geo_krs_Hamburg                  0
geo_krs_Leipzig                  0
geo_krs_M?hldorf_am_Inn_Kreis    0
geo_krs_M?nchen                  0
geo_krs_Mannheim                 0
dtype: int64

In [48]:
#concating ordinals and nominals
categorical = pd.concat([ordinals,nominals], axis=1).reset_index()

In [49]:
categorical.isna().sum()

level_0                          0
index                            0
energyefficiencyclass            0
balcony                          0
haskitchen                       0
cellar                           0
lift                             0
garden                           0
petsallowed_negotiable           0
petsallowed_no                   0
petsallowed_yes                  0
geo_krs_Berlin                   0
geo_krs_Bonn                     0
geo_krs_D?sseldorf               0
geo_krs_Dresden                  0
geo_krs_Frankfurt_am_Main        0
geo_krs_G?tersloh_Kreis          0
geo_krs_Hamburg                  0
geo_krs_Leipzig                  0
geo_krs_M?hldorf_am_Inn_Kreis    0
geo_krs_M?nchen                  0
geo_krs_Mannheim                 0
dtype: int64

In [50]:
categorical=categorical.drop(['level_0', 'index'], axis=1)

In [51]:
categorical=categorical.reset_index()

In [52]:
numericals=numericals.reset_index()

In [53]:
numericals

,index,baserent,noparkspaces,livingspace,norooms,floor
0,0,1680.00,1.0,91.00,1.5,2.0
1,1,1337.22,1.0,116.28,3.0,1.0
2,2,1321.61,1.0,104.89,4.0,4.0
3,3,1744.26,1.0,130.00,3.0,8.0
4,4,890.00,1.0,64.40,2.0,4.0
...,...,...,...,...,...,...
947,966,1115.00,1.0,103.14,4.0,3.0
948,967,1217.00,1.0,109.10,4.0,2.0
949,968,1159.36,2.0,144.92,4.0,1.0
950,969,1533.92,1.0,121.74,5.0,2.0


In [54]:
#concating categoricals and numericals
final_df = pd.concat([numericals,categorical], axis=1).drop('index', axis=1)

In [55]:
final_df

,baserent,noparkspaces,livingspace,norooms,floor,energyefficiencyclass,balcony,haskitchen,cellar,lift,...,geo_krs_Bonn,geo_krs_D?sseldorf,geo_krs_Dresden,geo_krs_Frankfurt_am_Main,geo_krs_G?tersloh_Kreis,geo_krs_Hamburg,geo_krs_Leipzig,geo_krs_M?hldorf_am_Inn_Kreis,geo_krs_M?nchen,geo_krs_Mannheim
0,1680.00,1.0,91.00,1.5,2.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,1337.22,1.0,116.28,3.0,1.0,3.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,1321.61,1.0,104.89,4.0,4.0,2.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,1744.26,1.0,130.00,3.0,8.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,890.00,1.0,64.40,2.0,4.0,2.0,0.0,1.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,1115.00,1.0,103.14,4.0,3.0,2.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,1,0,0,0
948,1217.00,1.0,109.10,4.0,2.0,2.0,0.0,1.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
949,1159.36,2.0,144.92,4.0,1.0,2.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
950,1533.92,1.0,121.74,5.0,2.0,3.0,0.0,0.0,1.0,0.0,...,0,0,1,0,0,0,0,0,0,0


In [57]:
final_df.columns

Index(['baserent', 'noparkspaces', 'livingspace', 'norooms', 'floor',
       'energyefficiencyclass', 'balcony', 'haskitchen', 'cellar', 'lift',
       'garden', 'petsallowed_negotiable', 'petsallowed_no', 'petsallowed_yes',
       'geo_krs_Berlin', 'geo_krs_Bonn', 'geo_krs_D?sseldorf',
       'geo_krs_Dresden', 'geo_krs_Frankfurt_am_Main',
       'geo_krs_G?tersloh_Kreis', 'geo_krs_Hamburg', 'geo_krs_Leipzig',
       'geo_krs_M?hldorf_am_Inn_Kreis', 'geo_krs_M?nchen', 'geo_krs_Mannheim'],
      dtype='object')

# Model creation
## X-y split (y is the target variable, in this case, "base rent")

In [ ]:
#X-y split
X=final_df.drop('baserent', axis=1)
y=final_df[['baserent']]

In [ ]:
X

In [ ]:
y

## train_test_split

In [ ]:
#train_test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [ ]:
X_train

In [ ]:
#X_train.hist()

In [ ]:
y_train.hist()

## using powertransform for X and Y


In [ ]:
from sklearn.preprocessing import PowerTransformer

In [ ]:
#fitting power transformer to use in train and test sets

pt=PowerTransformer()
pt.fit(X_train)

In [ ]:
#transforming train and test sets

X_train=pt.transform(X_train)
X_test=pt.transform(X_test)

In [ ]:
#transforming y

#pt.fit(y_train)
#y_train=pt.transform(y_train)
#y_test=pt.transform(y_test)

# CREATING LINEAR REGRESSION

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score

In [ ]:
lr=LinearRegression()

In [ ]:
#creating model
model=lr.fit(X_train,y_train)
lr.intercept_

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df['livingspace'], df['price'])
plt.xlabel('Living Space')
plt.ylabel('Price')
plt.title('Scatter Plot: Living Space vs Price')
plt.show()


In [ ]:
#visualizing
feature_names1=list(X.columns)
coefficients1 = model.coef_
coefficients1=[i for i in coefficients1[0]]
coefficients1=sorted(coefficients1)
fi_dict1 = {"name":feature_names1,"coeff":coefficients1}
fi_df1 = pd.DataFrame(fi_dict1)
fi_df1
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 30))
sns.barplot(y = fi_df1["name"],x =fi_df1["coeff"])

# checking model quality

In [ ]:
#test model
y_pred=y_pred=pd.DataFrame(lr.predict(X_test))

In [ ]:
#calculating mean errors
MSE_full=mse(y_pred,y_test)
RMSE_full=mse(y_pred,y_test, squared=False)
print(f"mse_full:{MSE_full}\nRMSE:{RMSE_full}")

In [ ]:
#calculating R2 for predictions
r2_score(y_test,y_pred)

In [ ]:
#calculating r2_score for train set
y_train_pred=pd.DataFrame(lr.predict(X_train))
r2_score(y_train,y_train_pred)